In [0]:
#!pip install eli5

In [0]:
import pandas as pd
import numpy as np

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

from ast import literal_eval
from tqdm import tqdm_notebook

In [101]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix"

/content/drive/My Drive/Colab Notebooks/dw_matrix


In [102]:
ls

data/  HelloGithub.ipynb  LICENSE  matrix_one/  README.md


In [0]:
!git add matrix_one/day5.ipynb

In [0]:
df = pd.read_csv('data/men_shoes.csv', low_memory=False)

In [7]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension', 'ean', 'features',
       'flavors', 'imageurls', 'isbn', 'keys', 'manufacturer',
       'manufacturernumber', 'merchants', 'name', 'prices_amountmin',
       'prices_amountmax', 'prices_availability', 'prices_color',
       'prices_condition', 'prices_count', 'prices_currency',
       'prices_dateadded', 'prices_dateseen', 'prices_flavor', 'prices_issale',
       'prices_merchant', 'prices_offer', 'prices_returnpolicy',
       'prices_shipping', 'prices_size', 'prices_source', 'prices_sourceurls',
       'prices_warranty', 'quantities', 'reviews', 'sizes', 'skus',
       'sourceurls', 'upc', 'vin', 'websiteids', 'weight'],
      dtype='object')

In [0]:
def run_model(feautures, model=DecisionTreeRegressor(max_depth=5)):
  X = df[ feautures ].values
  y = df['prices_amountmin'].values

  scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error')

  return np.mean(scores), np.std(scores)

In [9]:
df['brand_cat'] = df['brand'].map(lambda x: str(x).lower()).factorize()[0]
run_model(['brand_cat'])

(-58.133398968282776, 4.206122611474276)

In [10]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(['brand_cat'], model)

(-57.31783843165656, 4.181246596160967)

In [13]:
df.features.head().values

array(['[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SERVUS BY HONEYWELL"]},{"key":"manufacturer_part_number","value":["ZSR101BLMLG"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SER

In [15]:
str_dict = '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SERVUS BY HONEYWELL"]},{"key":"manufacturer_part_number","value":["ZSR101BLMLG"]}]'

literal_eval(str_dict)[0]['value'][0]

'Men'

In [0]:
def parse_features(x):
  output_dict = {}
  if str(x) == 'nan': return output_dict

  features = literal_eval(x.replace('\\"','"'))
  
  for item in features:
    key = item['key'].lower().strip()
    value = item['value'][0].lower().strip()

    output_dict[key] = value

  return output_dict


#{'key': 'Gender', 'value': ['Men']} 
df['features_parsed'] = df['features'].map(parse_features)

In [17]:
keys = set()

df['features_parsed'].map(lambda x: keys.update(x.keys()))
len(keys)

476

In [18]:
df.features_parsed.head().values

array([{'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'}],
      dtype=object)

In [19]:
def get_name_feat(key):
  return 'feat_' + key


for key in tqdm_notebook(keys):
  df[get_name_feat(key)] = df.features_parsed.map(lambda feats: feats[key] if key in feats else np.nan)

In [20]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'feat_fits most screen size', 'feat_is waterproof', 'feat_chronograph',
       'feat_sku number', 'feat_is lined', 'feat_location - city/state',
       'feat_theme', 'feat_colour code', 'feat_vehicle type',
       'feat_expandable'],
      dtype='object', length=526)

In [21]:
df[ df['feat_athlete'].isnull()].shape

(18272, 526)

In [23]:
df.shape[0]

18280

In [24]:
df [ False == df['feat_athlete'].isnull()].shape[0]

8

In [26]:
df [ False == df['feat_athlete'].isnull()].shape[0]/df.shape[0] * 100

0.0437636761487965

In [0]:
keys_stat = {}
for key in keys:
  keys_stat[key] = df [ False == df[get_name_feat(key)].isnull()].shape[0]/df.shape[0] * 100

In [34]:
{k: v for k,v in keys_stat.items() if v > 30}

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [0]:
df['feat_brand_cat'] = df['feat_brand'].factorize()[0]
df['feat_color_cat'] = df['feat_color'].factorize()[0]
df['feat_gender_cat'] = df['feat_gender'].factorize()[0]
df['feat_manufacturer part number_cat'] = df['feat_manufacturer part number'].factorize()[0]
df['feat_material_cat'] = df['feat_material'].factorize()[0]


df['feat_sport_cat'] = df['feat_sport'].factorize()[0]
df['feat_style_cat'] = df['feat_style'].factorize()[0]

for key in keys:
  df[get_name_feat(key) + '_cat'] = df[get_name_feat(key)].factorize()[0]
  

In [40]:
df[df.brand == df.feat_brand][['brand', 'feat_brand']].head()

,brand,feat_brand
2163,adidas,adidas
2417,adidas,adidas
3021,adidas,adidas
3054,adidas,adidas
3629,totes,totes


In [41]:
df[df.brand != df.feat_brand][['brand', 'feat_brand']].head()

,brand,feat_brand
0,Josmo,josmo
1,Josmo,josmo
2,SERVUS BY HONEYWELL,servus by honeywell
3,SERVUS BY HONEYWELL,servus by honeywell
4,SERVUS BY HONEYWELL,servus by honeywell


In [0]:
df['brand'] = df['brand'].map(lambda x:str(x).lower())

In [50]:
df[df.brand == df.feat_brand].shape 

(8846, 531)

In [51]:
model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(['brand_cat'], model)

(-57.32800056735274, 4.223574789470636)

In [74]:
feats_cat = [x for x in df.columns if 'cat' in x ]
feats_cat

['categories',
 'brand_cat',
 'feat_catalog',
 'feat_shoe category',
 'feat_certifications and listings',
 'feat_multi pack indicator',
 'feat_clothing category',
 'feat_recommended location',
 'feat_location - country',
 'feat_fabrication',
 'feat_location - city/state',
 'feat_brand_cat',
 'feat_color_cat',
 'feat_gender_cat',
 'feat_manufacturer part number_cat',
 'feat_material_cat',
 'feat_sport_cat',
 'feat_style_cat',
 'feat_hairstyle_cat',
 'feat_weight_cat',
 'feat_elastic banded waist_cat',
 'feat_finish_cat',
 'feat_watch case shape_cat',
 'feat_metal type_cat',
 'feat_global composite sports type_cat',
 'feat_polarized_cat',
 'feat_handbag style_cat',
 'feat_shoe closure_cat',
 'feat_dial material_cat',
 'feat_reinforced knee_cat',
 'feat_impact resistant_cat',
 'feat_sock style_cat',
 'feat_color mapping_cat',
 'feat_age start_cat',
 'feat_sku_cat',
 'feat_instrument_cat',
 'feat_cleaning, care & maintenance_cat',
 'feat_catalog_cat',
 'feat_domestic shipping_cat',
 'feat_

In [0]:
feats = ['brand_cat', 'feat_metal type_cat', 'feat_brand_cat', 'feat_gender_cat', 'feat_material_cat', 'feat_adjustable_cat', 'feat_case thickness_cat']
#feats += feats_cat
#feats = list(set(feats))


model = RandomForestRegressor(max_depth=5, n_estimators=100)
result = run_model(feats, model)

In [95]:
X = df[ feats ].values
y = df['prices_amountmin'].values

m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X, y)

print(result)
perm = PermutationImportance(m, random_state=1).fit(X, y);
eli5.show_weights(perm, feature_names=feats)

(-57.34687683012315, 4.095484635306737)


Weight,Feature
0.2539 ± 0.0097,brand_cat
0.1099 ± 0.0153,feat_material_cat
0.0286 ± 0.0058,feat_adjustable_cat
0.0182 ± 0.0016,feat_brand_cat
0.0170 ± 0.0013,feat_case thickness_cat
0.0065 ± 0.0012,feat_metal type_cat
0.0058 ± 0.0007,feat_gender_cat


In [61]:
df["brand"].value_counts(normalize=True)

nike                     0.097210
puma                     0.033315
ralph lauren             0.028775
vans                     0.021116
new balance              0.020295
                           ...   
stackable expressions    0.000055
o'neal                   0.000055
john ashford             0.000055
peterluckyi              0.000055
bally                    0.000055
Name: brand, Length: 1732, dtype: float64

In [69]:
df[df['brand'] == 'nike'].features_parsed.sample(5).values

array([{'season': 'all-season', 'material': 'canvas', 'gender': 'men', 'shoe size': '7.5', 'size': '7.5', 'color': 'red', 'model': '616575-641', 'manufacturer part number': '616575-641', 'brand': 'nike', 'age group': 'adult'},
       {'brand': 'nike', 'fabric content': '100% suede', 'color': 'brq brown/white-gm drk brwn'},
       {},
       {'gender': 'men', 'size': 'adult 10.5', 'clothing size': 'adult 10.5', 'brand': 'nike', 'color': 'gray/volt'},
       {'sport': 'running shoes', 'material': 'suede', 'gender': 'men', 'shoe size': '8 men us', 'size': '8', 'shoe category': 'running shoes;training shoes;lifestyle;casual shoes', 'assembled product dimensions (l x w x h)': '13.00 x 8.00 x 4.00 inches', 'color': 'blue', 'model': '684773 404', 'casual & dress shoe style': 'running shoes;training shoes;lifestyle;casual shoes', 'manufacturer part number': '684773 404', 'variant group id': '224#mp#684773 404', 'brand': 'nike', 'age group': 'men', 'fabric content': 'suede', 'occasion': 'runnin

In [70]:
df['feat_age group'].value_counts()

adult               4563
men                  350
child                 77
men's                 33
unisex                 6
toddler                4
mens                   4
infant                 4
boys'                  3
men||women             2
youth                  2
women                  2
women ,�� unisex       2
12 up                  1
adult ,�� teen         1
Name: feat_age group, dtype: int64